In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from sentence_transformers import SentenceTransformer, util
import torch
from dotenv import load_dotenv
from mongo_wrapper.mongo_wrapper import MongoWrapper

# states to categorize
STATES = [
    "Guanajuato",
    "Michoacán",
    "Sinaloa",
    "Chihuahua",
    "Guerrero",
    "Tamaulipas",
    "Baja California",
    "Zacatecas",
    "Colima",
    "Jalisco",
    "Aguascalientes",
    "Baja California Sur",
    "Campeche",
    "Coahuila",
    "Durango",
    "Hidalgo",
    "Estado de México",
    "Ciudad de México",
    "Morelos",
    "Nayarit",
    "Nuevo León",
    "Oaxaca",
    "Puebla",
    "Querétaro",
    "Quintana Roo",
    "San Luis Potosí",
    "Sonora",
    "Tabasco",
    "Tlaxcala",
    "Veracruz",
    "Yucatán",
    "Chiapas"]


# channels to analyze 
TARGET_CHANNELS = [
    "elpaismexico_2020",
    "ElUniversalOnline_2020",
    "proceso_unofficial_2020",
    "politicomx_2020",
    "lajornada_unofficial_2020",
    "larazondemexico_2020",
    "sinembargomx_2020",
    "elpaisamerica_2020"]

# define dimensions of poverty 
POVERTY_DIMENSIONS = {
    "INCOME": """
    desempleo salario mínimo bajos ingresos deudas familiares pobreza laboral
    pérdida de empleo ingreso insuficiente precariedad laboral empleo informal
    falta de oportunidades laborales reducción de salario inestabilidad económica
    recesión subempleo despidos masivos contratos temporales informalidad
    costos de vida elevados falta de empleo formal insuficiencia salarial
    """,

    "ACCESS TO HEALTH SERVICES": """
    falta de acceso a servicios de salud hospitales saturados escasez de medicamentos 
    deficiencias en la atención médica carencia de personal médico emergencia sanitaria
    costos elevados de tratamientos cierre de centros de salud lista de espera prolongada
    equipos médicos inoperantes desabasto de vacunas falta de atención especializada
    """,

    "EDUCATIONAL_LAG": """
    deserción escolar suspensión de clases carencia de docentes  
    dificultades de acceso a la educación educación interrumpida rezago académico 
    falta de recursos escolares acceso desigual a la educación deficiencias en formación 
    básica carencia de materiales educativos 
    """,

    "ACCESS TO SOCIAL SECURITY": """
    empleo informal ausencia de prestaciones sociales falta de contrato laboral 
    exclusión del sistema de pensiones carencia de protección social trabajo precario 
    derechos laborales no garantizados falta de cotización al sistema desprotección estructural
    dificultades para acceder al seguro social informalidad laboral empleo sin afiliación
    """,

    "HOUSING": """
    vivienda precaria hacinamiento falta de servicios básicos 
    infraestructura deteriorada zonas marginadas viviendas inseguras
    """,
    
    "ACCESS TO FOOD": """
    inseguridad alimentaria acceso limitado a alimentos inflación precios
    raciones insuficientes pobreza alimentaria aumento de precios comeder comunitario
    canasta básica crisis alimentaria comida pobre ayuda alimentaria 
    insuficiencia nutricional alimentación deficiente encarecimiento de alimentos
    inflación en alimentos carencia alimentaria productos básicos banco de alimentos
    alimentos inaccesibles gasto alimentario elevado programas alimentarios
    """,

    "SOCIAL_COHESION": """
    discriminación étnica marginación social exclusión comunidades vulnerables
    conflictos intercomunitarios tensiones sociales barreras sociales 
    desigualdad aislamiento social
    """}


class PovertyDimensionClassifier:
    def __init__(self):
        # sentence embedding
        self.model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')
        
        # create embeddings for poverty dimensions
        self.dimension_names = list(POVERTY_DIMENSIONS.keys())
        self.dimension_embeddings = self.model.encode(list(POVERTY_DIMENSIONS.values()), convert_to_tensor=True)
    
    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        text = re.sub(r'<.*?>', ' ', text)
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'[^\w\sáéíóúüñÁÉÍÓÚÜÑ]', ' ', text)
        return re.sub(r'\s+', ' ', text).strip().lower()
    
    # classify the text into a poverty dimension using sentence embeddings
    def classify_text(self, text, threshold=0.10):
        if not text:
            return None, 0.0
        
        cleaned_text = self.clean_text(text)
        if len(cleaned_text) < 10:  # avoid too short texts
            return None, 0.0
        
        text_embedding = self.model.encode(cleaned_text, convert_to_tensor=True)
        
        # compute cosine similarity
        cosine_scores = util.cos_sim(text_embedding, self.dimension_embeddings)[0]
        
        # find dimension with highest similarity 
        max_idx = torch.argmax(cosine_scores).item()
        max_score = cosine_scores[max_idx].item()
        if max_score >= threshold:
            return self.dimension_names[max_idx], max_score
        else:
            return None, max_score

# load only data of interest from MongoDB
def load_state_posts():
    
    MONGO_IP = os.getenv("MONGO_IP")
    MONGO_PORT = os.getenv("MONGO_PORT")
    MONGO_DB = os.getenv("MONGO_DB")
    MONGO_USERNAME = os.getenv("MONGO_USERNAME")
    MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")
    
    mongo_client = MongoWrapper(
        db=MONGO_DB,
        user=MONGO_USERNAME,
        password=MONGO_PASSWORD,
        ip=MONGO_IP,
        port=MONGO_PORT)
    
    all_channels = mongo_client.get_all_collections()
    available_target_channels = [channel for channel in TARGET_CHANNELS if channel in all_channels]
    
    # initialize a dictionary to store posts for each state
    state_posts = {state: [] for state in STATES}
    
    # classify posts by using regex patterns - if a post contains a state name, it will be classified as that state
    state_patterns = {state: re.compile(r'\b' + re.escape(state) + r'\b', re.IGNORECASE) for state in STATES}
    
    for channel in tqdm(available_target_channels, desc="loading channels"):
        posts = mongo_client.get_collection_entries(collection=channel)
        
        print(f"channel: {channel} - {len(posts)} post found")
        
        for post in tqdm(posts, desc=f"analysis {channel}", leave=False):
            post_text = post.get('text', '')
                
            for state, pattern in state_patterns.items():
                if pattern.search(post_text):
                    state_posts[state].append({
                        'text': post_text,
                        'author': post.get('author', ''),
                        'posting_ts': post.get('posting_ts', ''),
                        'channel': channel})
    
    for state in STATES:
        state_posts[state] = pd.DataFrame(state_posts[state])
    
    return state_posts

# classify posts by dimensions of poverty 
def analyze_poverty_dimensions(state_posts):
    classifier = PovertyDimensionClassifier()
    
    results = []

    for state, df in state_posts.items():
        print(f"\nanalyzing {state} ({len(df)} posts)...")
    
        dimension_counts = {dim: 0 for dim in POVERTY_DIMENSIONS.keys()}
        dimension_counts["OTHER"] = 0  # other types of posts 
    
        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Classifying {state}"):
            text = row['text']
        
            dimension, score = classifier.classify_text(text)
        
            if dimension:
                dimension_counts[dimension] += 1
            else:
                dimension_counts["OTHER"] += 1
        
        total_posts = len(df)
        dimension_percentages = {dim: (count / total_posts) * 100 for dim, count in dimension_counts.items()}
        
        print(f"\nresults for {state}:")
        print(f"total posts: {total_posts}")
        print("\ndistribution of posts across dimensions of poverty:")
        
        for dim, count in dimension_counts.items():
            dim_name = dim if dim != "OTHER" else "non-poverty posts"
            pct = dimension_percentages[dim]
            print(f"- {dim_name}: {count} post ({pct:.1f}%)")
        
        for dim in list(POVERTY_DIMENSIONS.keys()) + ["OTHER"]:
            results.append({
                'state': state,
                'dimension': dim,
                'count': dimension_counts[dim],
                'percentage': dimension_percentages[dim],
                'total_posts': total_posts})
    
    results_df = pd.DataFrame(results)
    return results_df

def main():
    state_posts = load_state_posts()
    
    results = analyze_poverty_dimensions(state_posts)
    
    results.to_csv("tg_results_2020.csv", index=False)
    
    pivot_counts = results.pivot(index='state', columns='dimension', values='count')
    print("\ncount of posts per dimension:")
    print(pivot_counts)
    
    pivot_percentages = results.pivot(index='state', columns='dimension', values='percentage')
    print("\npercentage of posts per dimension:")
    print(pivot_percentages.round(1))

if __name__ == "__main__":
    main()

2025-05-24 23:41:50,056 WARNING Logger Mongo was configured with True console stream
2025-05-24 23:41:50,443 INFO Connected to thesis database on 206.81.16.39
loading channels:   0%|          | 0/8 [00:00<?, ?it/s]

channel: elpaismexico_2020 - 0 post found


channel: ElUniversalOnline_2020 - 1424 post found


loading channels:  25%|██▌       | 2/8 [00:01<00:03,  1.71it/s]

channel: proceso_unofficial_2020 - 0 post found


...


results for Veracruz:
total posts: 92

distribution of posts across dimensions of poverty:
- INCOME: 12 post (13.0%)
- ACCESS TO HEALTH SERVICES: 37 post (40.2%)
- EDUCATIONAL_LAG: 5 post (5.4%)
- ACCESS TO SOCIAL SECURITY: 32 post (34.8%)
- HOUSING: 1 post (1.1%)
- ACCESS TO FOOD: 2 post (2.2%)
- SOCIAL_COHESION: 1 post (1.1%)
- non-poverty posts: 2 post (2.2%)

analyzing Yucatán (42 posts)...


Classifying Yucatán: 100%|██████████| 42/42 [00:01<00:00, 38.20it/s]



results for Yucatán:
total posts: 42

distribution of posts across dimensions of poverty:
- INCOME: 7 post (16.7%)
- ACCESS TO HEALTH SERVICES: 18 post (42.9%)
- EDUCATIONAL_LAG: 0 post (0.0%)
- ACCESS TO SOCIAL SECURITY: 14 post (33.3%)
- HOUSING: 1 post (2.4%)
- ACCESS TO FOOD: 1 post (2.4%)
- SOCIAL_COHESION: 0 post (0.0%)
- non-poverty posts: 1 post (2.4%)

analyzing Chiapas (81 posts)...


Classifying Chiapas: 100%|██████████| 81/81 [00:02<00:00, 36.05it/s]



results for Chiapas:
total posts: 81

distribution of posts across dimensions of poverty:
- INCOME: 7 post (8.6%)
- ACCESS TO HEALTH SERVICES: 40 post (49.4%)
- EDUCATIONAL_LAG: 9 post (11.1%)
- ACCESS TO SOCIAL SECURITY: 17 post (21.0%)
- HOUSING: 1 post (1.2%)
- ACCESS TO FOOD: 3 post (3.7%)
- SOCIAL_COHESION: 1 post (1.2%)
- non-poverty posts: 3 post (3.7%)

count of posts per dimension:
dimension            ACCESS TO FOOD  ACCESS TO HEALTH SERVICES  \
state                                                            
Aguascalientes                    0                          4   
Baja California                   0                         12   
Baja California Sur               0                          4   
Campeche                          2                         10   
Chiapas                           3                         40   
Chihuahua                         8                         33   
Ciudad de México                  0                         18   
Coahuila   